**XS 0130 | Programación para Estadística II**

**Proyecto 01**

**Estudiantes:**
Fabiola Fang, Catalina Monge & Marisol Quesada

**Tema:**
Simulador de una red social

In [ ]:
import pandas as pd
import random

In [ ]:
# Elementos fundamentales de una red social: usuarios, publicaciones y la capaciad de interactuar mediante comentarios

# Usuarios

class Usuario:
    def __init__(self, username):
        self.username = username
        self.amix = set() #para que no se repitan
        self.posteos = [] #en una lista para que funcione como objeto modificabke
    def __str__(self):
        return f"Usuario: {self.username} | Amistades: {len(self.amix)} | Publicaciones: {len(self.posteos)}"

# Antes de definir la capacidad de comentar, se determina cómo se espera que sea un comentario


class Comentario:
    def __init__(self, username, mensaje):
        self.username = username
        self.mensaje = mensaje
    def __str__(self):
        return f"{self.username}: {self.mensaje}"

# Publicaciones

class Post:
    def __init__(self, id_pub, username, contenido):
        self.id = id_pub # según df generado
        self.username = username
        self.contenido = contenido
        self.likes = set() #para que no se pueda dar más de un like
        self.comentarios = [] #nuevamente propiedad mutable de la lista
    def likear(self, username): 
        self.likes.add(username) #añade likes contemplando quién lo dio para no repitir
    def comentar(self, comentario):
        self.comentarios.append(comentario) #añade comentarios

    def __str__(self):
        return f"({self.id}) {self.username}: {self.contenido} | Likes: {len(self.likes)} | Comentarios: {len(self.comentarios)}"

#esta última devuelve un resumen por publicación

In [ ]:
# Importar la base de datos: creada artificialmente y posteriormente personalizada

# Se realiza como una función para poder añadir comandos que permitan manipular la base y no solo cargarla

def importar(archivo_excel):
    df = pd.read_excel(archivo_excel) #se pone el nombre del archivo como argumento
    users_df = {} #más eficiente al asociarse a una clave única, previene duplicados
    post_df = {}
    contador_posts = 0 #vacio

    # Crear usuarios
    for usuario in df["usuario"].dropna().unique(): #verficar que todos los usuarios sean únicos y no hayan comments/post sin autor
        if usuario not in users_df:
            users_df[usuario] = Usuario(usuario) #la posibilidad de crear un perfil nuevo

    # Crear publicaciones
    publicaciones = df[df["tipo"] == "publicacion"]
    for _, row in publicaciones.iterrows(): #procesar cada fila individualmente 
        pub = Post(row["id"], row["usuario"], row["contenido"])
        post_df[pub.id] = pub #se agrega publicación con respectivo id al df
        users_df[row["usuario"]].posteos.append(pub) #se le asigna el usuario
        contador_posts = max(contador_posts, row["id"]) #actualiza el contador del id

    # Crear amistades
    amistades = df[df["tipo"] == "amigo"]
    for _, row in amistades.iterrows():
        u1, u2 = row["usuario"], row["amigo"] #comparar dos usuarios 
        if u1 in users_df and u2 in users_df:
            users_df[u1].amix.add(users_df[u2])
            users_df[u2].amix.add(users_df[u1])

    # Agregar comentarios
    comentarios = df[df["tipo"] == "comentario"]
    for _, row in comentarios.iterrows():
        comentario = Comentario(row["usuario"], row["contenido"])
        if row["pub_id"] in post_df:
            post_df[row["pub_id"]].comentar(comentario) 

    # Agregar likes
    likes = df[df["tipo"] == "like"]
    for _, row in likes.iterrows():
        if row["pub_id"] in post_df:
            post_df[row["pub_id"]].likear(row["usuario"])

    return users_df, post_df, contador_posts

In [1]:
# Funciones para el menú interactivo


def menu_usuario(persona, post_df):
    print(f"\n~~~ Este es el perfil del usuario {persona.username} ~~~ ")
    print("Amistades")
    for x in persona.amix:
        print(" -", x.username)
    print("Publicaciones")
    for i in persona.posteos:
        print(" -", i)
    print("Comentarios recibidos:")
    for p in persona.posteos:
        for c in p.comentarios: 
            print(f"En la publicación ({p.id}): se han recibido los comentarios {c}")
    print("Likes recibidos:")
    for l in persona.posteos:
        print(f"La publicación ({l.id}): tiene {len(l.likes)} likes")


def menu_registrar_usuario(users_df):
    nombre = input("Ingresa tu nombre de usuario")
    if nombre in users_df:
        print("Esta cuenta ya existe") #verifica si ya existe. Si no, da la posibilidad de creardo
        return
    users_df[nombre] = Usuario(nombre)
    print(f"Usuario {nombre} registrado con éxito")

def menu_crear_publicacion(persona, post_df, contador_posts):
    post = input("¿Qué quieres publicar?")
    contador_posts += 1
    p = Post(contador_posts, persona.username, post) #clase arriba
    post_df[contador_posts] = p #asigna post a todoas las pubicaciones en general
    persona.posteos.append(p) #identifca el quien
    print("Publicación realizada")
    return contador_posts #para que se pueda volver a utilizar

def menu_dar_like(persona, post_df):
    print("¿A qué publicación le quieres dar like?")
    totalidad_publicaciones = list(post_df.values()) #saca los elementos del dicccionario y los coloca en una lista para iterarlos
    for p in totalidad_publicaciones:
        print(p)
    pub_id = int(input("Escribe el ID de la publicación a la que quieres dar like: ")) #como cada posteo tiene ID, que señale cual
    if pub_id in post_df:
        pub = post_df[pub_id] 
        if persona.username in pub.likes: 
            print("Ya has dado like anteriormente")
        else:
            pub.likear(persona.username)
            print("Le has dado like")
    else:
        print("Este ID de publicación no existe")


def menu_comentar_publicacion(persona, post_df):
    print("¿Qué publicación quieres comentar?")
    totalidad_publicaciones = list(post_df.values())
    for p in totalidad_publicaciones:
        print(p)
    pub_id = int(input("Escribe el ID de la publicación a la que quieres comentar: "))
    if pub_id in post_df:
        texto = input("Escribe tu comentario: ")
        comentario = Comentario(persona.username, texto)
        post_df[pub_id].comentar(comentario)
        print("Comentario agregado")
    else:
        print("Este ID de publicación no existe")


def menu_ver_feed(persona):
    feed = []
    for x in persona.amix:
        feed.extend(x.posteos)
    feed.sort(key=lambda p: p.id, reverse=True)  # Para que salgan primero las más recientes
    print(f"\n ~~~ Feed para el perfil de {persona.username} ~~~")
    if not feed:
        print("No hay publicaciones recientes")
    for pub in feed:
        print(pub)
        for c in pub.comentarios: #si hay comentarios en una publicación mostrada, también se despliegan
            print(f"{c}")


def menu_recomendar_amigos(persona, users_df):
    recomendaciones = set() #igual que no se repita

    # Caso 1: si tiene amigos: recomendar amigos de amigos
    if persona.amix:
        for amigo in persona.amix:
            for fof in amigo.amix:
                if (fof != persona and fof not in persona.amix):
                    recomendaciones.add(fof) #revisar la lista de amigos y brinda las sugerenciss 

    # Caso 2: si NO tiene amigos: sugerir 3 usuarios aleatorios
    else:
        alternativas = [x for x in users_df.values() if x != persona]
        recomendaciones = set(random.sample(alternativas, min(3, len(alternativas))))

    if recomendaciones:
        print("Personas que quizás conozcas")
        for x in recomendaciones:
            print(f" - {x.username}")
    else:
        print("No hay recomendaciones disponibles")

In [ ]:
def menu_agregar_amigo(persona, users_df):
    print("\n~~~ Agrega amistades ~~~")
    print("Personas que quizás conozcas")
    for nombre in users_df:
        if nombre != persona.username and users_df[nombre] not in persona.amix:
            print(" -", nombre)
    
    quien = input("Escribe el nombre del usuario que quieres agregar como amistad: ")
    
    if quien not in users_df:
        print("Usuario no encontrado")
        return
    
    amigo_deseado = users_df[quien]
    
    if amigo_deseado in persona.amix:
        print("Ya son amigos.")
        return
    
    persona.amix.add(amigo_deseado)
    amigo_deseado.amix.add(persona)
    print(f"{amigo_deseado.username} ahora son amigos")

In [ ]:
# Menú

def menu(users_df, post_df, contador_posts):
    nombre_usuario = input("Ingresa tu nombre de usuario para iniciar sesión: ")
    usuario = users_df.get(nombre_usuario, None)
    
    if usuario is None:
        print("Usuario no encontrado. Regístrate primero en el menú.")


    while True:
        print("\n ~~~ Twitter ~~~")
        print("1. Registrárse")
        print("2. Perfil")
        print("3. Feed")
        print("4. Twittear")
        print("5. Fav")
        print("6. Comentar tweet")
        print("7. Seguir")
        print("8. Recomendar seguidores")
        print("9. Salir")
        opcion = input("¿Qué deseas hacer?: ")

        if opcion == "1":
            menu_registrar_usuario(users_df)
            nombre_usuario = input("Ingresa tu nombre de usuario para iniciar sesión: ")
            user = users_df.get(nombre_usuario, None)
        elif opcion == "2":
            menu_usuario(user, post_df)
        elif opcion == "3":
            menu_ver_feed(user)
        elif opcion == "4":
            contador_posts = menu_crear_publicacion(user, post_df, contador_posts)
        elif opcion == "5":
            menu_dar_like(user, post_df)
        elif opcion == "6":
            menu_comentar_publicacion(user, post_df)
        elif opcion == "7" and user:
            menu_agregar_amigo(user, users_df)
        elif opcion == "8":
            menu_recomendar_amigos(user, users_df)
        elif opcion == "9":
            print("Cerrar sesión")
            break
        else:
            print("Opción inválida. Verifica indicación o inicia sesión primero")

In [ ]:
# Ejecución final 

if __name__ == "__main__":
    archivo = "red_social_editado.xlsx"
    users_df, post_df, contador_posts = importar(archivo)
    menu(users_df, post_df, contador_posts)